# Fragmentation for train data
Aquesta part del codi la fem perquè hem trobat problemes per pujar les dades al repositori  

In [2]:
num_art = 10
# Numero d'artistes a agafar

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
from random import seed # for setting seed
import tensorflow
from IPython import sys_info
import os
import gc # garbage collection
import shutil

In [3]:
directori_info = '../../all_data_info.csv'
df = pd.read_csv(directori_info, sep=',')
df

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
103245,Jackson Pollock,1948.0,abstract,682.0,220.0,96405.0,wikiart,Action painting,Number 13A (Arabesque),train_and_test,True,25525.jpg
103246,Bernardo Strozzi,NaN,religious painting,329.0,456.0,127594.0,wikiart,Baroque,St. Francis of Assisi,train_only,True,47038.jpg
103247,Josef Sima,NaN,landscape,293.0,512.0,102519.0,wikiart,Surrealism,Maisons à la campagne II,train_and_test,False,7680.jpg
103248,Brett Whiteley,1982.0,marina,293.0,512.0,167423.0,wikiart,NaN,Thebe's Revenge,train_and_test,True,9021.jpg


In [4]:
artist_dict = df['artist'].value_counts().to_dict()
artist_dict

{'Ivan Aivazovsky': 500,
 'John Singer Sargent': 500,
 'Pierre-Auguste Renoir': 500,
 'Marc Chagall': 500,
 'Pablo Picasso': 500,
 'Zdislav Beksinski': 500,
 'Ivan Shishkin': 500,
 'Gustave Dore': 500,
 'Ilya Repin': 500,
 'Giovanni Battista Piranesi': 500,
 'Albrecht Durer': 500,
 'Rembrandt': 500,
 'Paul Cezanne': 499,
 'Pyotr Konchalovsky': 499,
 'Camille Pissarro': 499,
 'Martiros Saryan': 499,
 'Boris Kustodiev': 498,
 'Paul Gauguin': 498,
 'Claude Monet': 498,
 'Raphael Kirchner': 497,
 'Eugene Boudin': 496,
 'Nicholas Roerich': 496,
 'Henri Matisse': 495,
 'Edgar Degas': 495,
 'Vincent van Gogh': 494,
 'Childe Hassam': 493,
 'Camille Corot': 488,
 'Salvador Dali': 485,
 'Theophile Steinlen': 485,
 'Alfred Sisley': 464,
 'Odilon Redon': 451,
 'Isaac Levitan': 449,
 'Fernand Leger': 437,
 'James Tissot': 427,
 'Eyvind Earle': 422,
 'Zinaida Serebriakova': 412,
 'Henri Martin': 405,
 'Ernst Ludwig Kirchner': 388,
 'Francisco Goya': 386,
 'Maurice Prendergast': 379,
 'Peter Paul Rub

In [5]:
df['artist'].nunique()

2319

Per tant hi ha 2319 artistes diferents, i també sabem el numero de vegades que apareix cada artista

In [6]:
artist_dict = dict(sorted(artist_dict.items(), key=lambda item: item[1], reverse=True))
artist_dict

{'Ivan Aivazovsky': 500,
 'John Singer Sargent': 500,
 'Pierre-Auguste Renoir': 500,
 'Marc Chagall': 500,
 'Pablo Picasso': 500,
 'Zdislav Beksinski': 500,
 'Ivan Shishkin': 500,
 'Gustave Dore': 500,
 'Ilya Repin': 500,
 'Giovanni Battista Piranesi': 500,
 'Albrecht Durer': 500,
 'Rembrandt': 500,
 'Paul Cezanne': 499,
 'Pyotr Konchalovsky': 499,
 'Camille Pissarro': 499,
 'Martiros Saryan': 499,
 'Boris Kustodiev': 498,
 'Paul Gauguin': 498,
 'Claude Monet': 498,
 'Raphael Kirchner': 497,
 'Eugene Boudin': 496,
 'Nicholas Roerich': 496,
 'Henri Matisse': 495,
 'Edgar Degas': 495,
 'Vincent van Gogh': 494,
 'Childe Hassam': 493,
 'Camille Corot': 488,
 'Salvador Dali': 485,
 'Theophile Steinlen': 485,
 'Alfred Sisley': 464,
 'Odilon Redon': 451,
 'Isaac Levitan': 449,
 'Fernand Leger': 437,
 'James Tissot': 427,
 'Eyvind Earle': 422,
 'Zinaida Serebriakova': 412,
 'Henri Martin': 405,
 'Ernst Ludwig Kirchner': 388,
 'Francisco Goya': 386,
 'Maurice Prendergast': 379,
 'Peter Paul Rub

In [7]:
artistes = list(artist_dict.keys())[:num_art]
artistes

['Ivan Aivazovsky',
 'John Singer Sargent',
 'Pierre-Auguste Renoir',
 'Marc Chagall',
 'Pablo Picasso',
 'Zdislav Beksinski',
 'Ivan Shishkin',
 'Gustave Dore',
 'Ilya Repin',
 'Giovanni Battista Piranesi']

In [64]:
df_filtered = df[df['artist'].isin(artistes)]
df_filtered

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
14,Ivan Aivazovsky,1873,marina,7444.0,5811.0,3189597.0,wikiart,Romanticism,Storm at sea,train_and_test,True,99442.jpg
28,Gustave Dore,1866,religious painting,4915.0,6068.0,29811319.0,wikiart,Romanticism,Death of Eleazer,train_and_test,True,7486.jpg
29,Gustave Dore,NaN,religious painting,4770.0,6219.0,32443139.0,wikiart,Romanticism,The-Deluge,train_and_test,True,35766.jpg
34,Gustave Dore,NaN,illustration,5878.0,4678.0,24158097.0,wikiart,Romanticism,The hoarders and wasters,train_and_test,False,31977.jpg
82,Ivan Aivazovsky,1850,marina,5815.0,3840.0,16019462.0,wikiart,Romanticism,The Ninth Wave,train_and_test,True,81750.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
85626,Zdislav Beksinski,NaN,symbolic painting,461.0,600.0,94258.0,wikiart,Surrealism,Untitled,train_and_test,False,98069.jpg
85633,Zdislav Beksinski,NaN,symbolic painting,461.0,600.0,45705.0,wikiart,Surrealism,Untitled,train_and_test,True,101416.jpg
85720,Zdislav Beksinski,NaN,symbolic painting,470.0,587.0,56827.0,wikiart,Surrealism,Untitled,train_and_test,True,62958.jpg
85789,Zdislav Beksinski,NaN,symbolic painting,459.0,600.0,89026.0,wikiart,Surrealism,Untitled,train_and_test,True,78737.jpg


In [65]:
file_list = df_filtered['new_filename'].tolist()
file_list

['99442.jpg',
 '7486.jpg',
 '35766.jpg',
 '31977.jpg',
 '81750.jpg',
 '85018.jpg',
 '71922.jpg',
 '96147.jpg',
 '11889.jpg',
 '856.jpg',
 '67325.jpg',
 '94943.jpg',
 '23452.jpg',
 '42432.jpg',
 '76388.jpg',
 '82140.jpg',
 '83092.jpg',
 '85670.jpg',
 '19571.jpg',
 '39086.jpg',
 '76577.jpg',
 '57901.jpg',
 '44603.jpg',
 '97939.jpg',
 '74498.jpg',
 '84023.jpg',
 '7022.jpg',
 '41368.jpg',
 '80565.jpg',
 '14108.jpg',
 '37308.jpg',
 '29842.jpg',
 '55911.jpg',
 '25140.jpg',
 '8328.jpg',
 '3016.jpg',
 '33696.jpg',
 '72281.jpg',
 '30287.jpg',
 '42728.jpg',
 '28918.jpg',
 '34080.jpg',
 '2186.jpg',
 '59420.jpg',
 '65797.jpg',
 '4155.jpg',
 '40565.jpg',
 '98150.jpg',
 '95183.jpg',
 '36944.jpg',
 '32755.jpg',
 '73244.jpg',
 '69250.jpg',
 '63452.jpg',
 '74387.jpg',
 '38337.jpg',
 '1155.jpg',
 '49948.jpg',
 '48500.jpg',
 '18290.jpg',
 '36739.jpg',
 '8568.jpg',
 '49442.jpg',
 '89963.jpg',
 '58805.jpg',
 '49185.jpg',
 '74871.jpg',
 '4613.jpg',
 '48554.jpg',
 '67208.jpg',
 '91362.jpg',
 '112.jpg',
 '834

In [69]:
directori = '../..'
content = os.listdir(directori)

for c in content:
    if "train_" in c:
        if "train_10_artists" != c:
            directori_folder = os.path.join(directori, c)
            if os.path.isdir(directori_folder):
                print(c)
                files = os.listdir(directori_folder)
                for foto in files:
                    if foto in file_list:
                        rute = os.path.join(directori_folder, foto)
                        rute_final = os.path.join('../../train_10_artists', foto)
                        shutil.move(rute, rute_final)
                        
# Això fa que les imatges passin a la nova carpeta

train_5
train_6


In [73]:
carpeta_destino = '../../train_10_artists'
files_final = os.listdir(carpeta_destino)
con = 0
for f in files_final:
    if f not in file_list:
        print(f)
        con += 1
print(con)

0


Aquest serà el conjunt a utilitzar com a train, però sense fer el Data Augmentation
## Aplicació del Data Augmentation: